<a href="https://colab.research.google.com/github/adithyaGHegde/Data-Science-ML-AI/blob/main/UniversityAdmissionPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from collections import defaultdict
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import neighbors, svm,metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('../input/velohackdsaisoc/train.csv')
labels = pd.read_csv('../input/velohackdsaisoc/labels.csv') # use these labels to label encode the university column. You can also use labelencoder from sklearn
test = pd.read_csv('../input/velohackdsaisoc/test.csv')
uids = test['uid'] # use this to first store uids. After doing this you can drop the uids column from the test dataset and perform predictions on it.

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
for col in test.columns:
    print(f"No. of unique entries in {col} column: {test[col].nunique()}")

In [ ]:
test_cpy = test.copy()
test_cpy.drop('greA',axis=1,inplace =True)

In [ ]:
# test.drop(['userName','uid'],axis=1,inplace = True)
# test.program = test.program.astype("category").cat.codes
# test.ugCollege = test.ugCollege.astype("category").cat.codes
# test.termAndYear = test.termAndYear.astype("category").cat.codes
# test.department = test.department.astype("category").cat.codes
# test.major = test.major.astype("category").cat.codes

# test.head()

In [ ]:
test.univName = test.univName.astype(str)

TargetEncoder requires the column to have string values to encode it based on the target column, so i made univName into string type instead of replacing it with labels.

In [ ]:
ugCollege_encoding = train.groupby(['ugCollege'])['admit'].mean().to_dict()
major_encoding = train.groupby(['major'])['admit'].mean().to_dict()
univName_encoding = train.groupby(['univName'])['admit'].mean().to_dict()
department_encoding = train.groupby(['department'])['admit'].mean().to_dict()
termAndYear_encoding = train.groupby(['termAndYear'])['admit'].mean().to_dict()

# train_new['ugCollegeEncoded'] = train['ugCollege'].map(ugCollege_encoding)
# train_new['majorEncoded'] = train['major'].map(species_encoding)
# train_new['univNameEncoded'] = train['univName'].map(species_encoding)
# train_newtrain_new['departmentEncoded'] = train['department'].map(species_encoding)
# train_new['termAndYearEncoded'] = train['termAndYear'].map(species_encoding)


test['ugCollege'] = train['ugCollege'].map(ugCollege_encoding)
test['major'] = train['major'].map(major_encoding)
test['univName'] = train['univName'].map(univName_encoding)
test['department'] = train['department'].map(department_encoding)
test['termAndYear'] = train['termAndYear'].map(termAndYear_encoding)

In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder()
test['ugCollege'] = encoder.fit_transform(test['ugCollege'], test['admit'])
test['major'] = encoder.fit_transform(test['major'], test['admit'])
test['univName'] = encoder.fit_transform(test['univName'], test['admit'])
test['department'] = encoder.fit_transform(test['department'], test['admit'])
test['termAndYear'] = encoder.fit_transform(test['termAndYear'], test['admit'])


TargetEncoded all the categorical columns.

In [ ]:
test.head()

,userName,major,researchExp,industryExp,toeflScore,program,department,internExp,greV,greQ,journalPubs,greA,termAndYear,confPubs,ugCollege,cgpa,univName,uid
0,blaze1990,0.490697,0,0,111.0,MS,0.596386,0.0,157,165,0,3.0,0.514531,0,0.882353,0.7560,0.471349,blaze1990University of Florida
1,raghu449,0.498195,0,0,105.0,MS,0.490394,0.0,164,159,0,2.5,0.508613,0,0.453901,0.6325,0.143050,raghu449Cornell University
2,varun1044,0.608409,0,0,101.0,MS,0.490394,0.0,35,50,0,5.0,0.508613,0,0.530831,0.7300,0.430079,varun1044Texas A and M University College Station
3,nileshshinde9999,0.452137,0,0,91.0,MS,0.501425,0.0,154,164,0,3.0,0.514531,0,0.666667,0.6100,0.739035,nileshshinde9999University of Colorado Boulder
4,subhajit22,0.490697,0,0,105.0,PhD,0.478261,0.0,155,163,0,3.5,0.516779,0,0.469274,0.8120,0.329897,subhajit22Columbia University


In [ ]:
test.drop(['userName','uid','greA'],inplace=True,axis=1)

Dropped the columns that are not needed

In [ ]:
test.columns

Index(['major', 'researchExp', 'industryExp', 'toeflScore', 'program',
       'department', 'internExp', 'greV', 'greQ', 'journalPubs', 'termAndYear',
       'confPubs', 'ugCollege', 'cgpa', 'univName'],
      dtype='object')

In [ ]:
cat_cols = ['major','program','department','termAndYear','ugCollege','univName']
num_cols = ['researchExp','industryExp','toeflScore','internExp','greV','greQ','journalPubs','confPubs','cgpa']

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
preprocessor1 = ColumnTransformer(
    transformers = [
        ('onehotcat', OneHotEncoder(), ['program']),
        ('num', MinMaxScaler(), ['toeflScore','greV','greQ','cgpa']),
    ],
    remainder = 'passthrough',
)
preprocessor2 = ColumnTransformer(
    transformers = [
#         ('pca', pca, num_cols),
        ('dropper', 'drop', ['prog_4'])
    ],
    remainder = 'passthrough',
)

I didn't run PCA, that could be tried too

In [ ]:
preprocessor1_features = ['prog_1','prog_2','prog_3','prog_4','toeflScore','greV','greQ','cgpa','major', 'researchExp', 'industryExp','department', 'internExp','journalPubs','termAndYear', 'confPubs', 'ugCollege', 'univName']
preprocessor2_features = ['prog_1','prog_2','prog_3','toeflScore','greV','greQ','cgpa','major', 'researchExp', 'industryExp','department', 'internExp','journalPubs','termAndYear', 'confPubs', 'ugCollege', 'univName']

In [ ]:
X = test.iloc[:,:]
# y = test['admit']
X.head()

,major,researchExp,industryExp,toeflScore,program,department,internExp,greV,greQ,journalPubs,termAndYear,confPubs,ugCollege,cgpa,univName
0,0.490697,0,0,111.0,MS,0.596386,0.0,157,165,0,0.514531,0,0.882353,0.7560,0.471349
1,0.498195,0,0,105.0,MS,0.490394,0.0,164,159,0,0.508613,0,0.453901,0.6325,0.143050
2,0.608409,0,0,101.0,MS,0.490394,0.0,35,50,0,0.508613,0,0.530831,0.7300,0.430079
3,0.452137,0,0,91.0,MS,0.501425,0.0,154,164,0,0.514531,0,0.666667,0.6100,0.739035
4,0.490697,0,0,105.0,PhD,0.478261,0.0,155,163,0,0.516779,0,0.469274,0.8120,0.329897


In [ ]:
X = pd.DataFrame(preprocessor1.fit_transform(X), columns=preprocessor1_features)
X = pd.DataFrame(preprocessor2.fit_transform(X), columns=preprocessor2_features)

In [ ]:
X.head(20)

,prog_1,prog_2,prog_3,toeflScore,greV,greQ,cgpa,major,researchExp,industryExp,department,internExp,journalPubs,termAndYear,confPubs,ugCollege,univName
0,0.0,1.0,0.0,0.091736,0.923529,0.962406,0.051932,0.490697,0.0,0.0,0.596386,0.0,0.0,0.514531,0.0,0.882353,0.471349
1,0.0,1.0,0.0,0.086777,0.964706,0.917293,0.043448,0.498195,0.0,0.0,0.490394,0.0,0.0,0.508613,0.0,0.453901,0.143050
2,0.0,1.0,0.0,0.083471,0.205882,0.097744,0.050146,0.608409,0.0,0.0,0.490394,0.0,0.0,0.508613,0.0,0.530831,0.430079
3,0.0,1.0,0.0,0.075207,0.905882,0.954887,0.041903,0.452137,0.0,0.0,0.501425,0.0,0.0,0.514531,0.0,0.666667,0.739035
4,0.0,0.0,0.0,0.086777,0.911765,0.947368,0.055779,0.490697,0.0,0.0,0.478261,0.0,0.0,0.516779,0.0,0.469274,0.329897
5,0.0,1.0,0.0,0.086777,0.952941,0.887218,0.051520,0.490697,0.0,0.0,0.527357,0.0,0.0,0.472155,0.0,0.600000,0.143050
6,0.0,1.0,0.0,0.091736,0.964706,0.969925,0.054954,0.452137,0.0,0.0,0.483974,0.0,0.0,0.514531,0.0,0.750000,0.471349
7,0.0,1.0,0.0,0.090083,0.923529,0.939850,0.056534,0.400810,0.0,0.0,0.525361,0.0,0.0,0.514531,0.0,0.527850,0.386667
8,0.0,1.0,0.0,0.085950,0.876471,0.954887,0.058458,0.452137,0.0,0.0,1.000000,0.0,0.0,0.508613,0.0,0.750000,0.333333
9,0.0,1.0,0.0,0.088430,0.917647,0.924812,0.058046,0.749659,0.0,0.0,0.576923,0.0,0.0,0.516779,0.0,0.537580,0.342895


This is what our data finally looks like

In [ ]:
corr = test_cpy.corr(method = 'spearman')
plt.figure(figsize=(20,6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='Blues')
plt.title('Spearman Correlation Heatmap')
plt.show()

In [ ]:
# pairplot_cols = num_cols
# pairplot_cols.append('admit')
# figure = plt.figure(figsize=(20,10))
# sns.pairplot(test[pairplot_cols], hue='admit', palette='GnBu')
# plt.show()

In [ ]:
# fig, axes = plt.subplots(9, 3, figsize=(20,40))
# for i, col in zip(range(9), num_cols):
#     sns.stripplot(ax=axes[i][0], x='admit', y=col, data=test, palette='GnBu', jitter=True)
#     axes[i][0].set_title(f'{col} Stripplot')
#     sns.histplot(ax=axes[i][1], x=col, data=test, kde=False, bins=10, palette='GnBu', hue='admit', multiple='dodge')
#     axes[i][1].set_title(f'{col} Displot')
#     sns.boxplot(ax=axes[i][2], x='admit', y=col, data=test, palette='GnBu', hue='admit')
#     axes[i][2].set_title(f'{col} Boxplot')

In [ ]:
# def outlier_limits( test, col_name, q1 = 0.25, q3 = 0.75):
#     quartile1 =  test[col_name].quantile(q1)
#     quartile3 =  test[col_name].quantile(q3)
#     interquantile_range = quartile3 - quartile1
#     up_limit = quartile3 + 1.5 * interquantile_range
#     low_limit = quartile1 - 1.5 * interquantile_range
#     return low_limit, up_limit

# def replace_with_limits(test, variable, q1 = 0.25, q3 = 0.75):
#     low_limit, up_limit = outlier_limits(test, variable, q1 = q1, q3 = q3)
#     test.loc[(test[variable] < low_limit), variable] = low_limit
#     test.loc[(test[variable] > up_limit), variable] = up_limit

# for variable in test[num_cols].columns:
#     replace_with_limits(test, variable)

In [ ]:
# fig, axes = plt.subplots(9, 3, figsize=(20,40))
# for i, col in zip(range(9), num_cols):
#     sns.stripplot(ax=axes[i][0], x='admit', y=col, data=test, palette='GnBu', jitter=True)
#     axes[i][0].set_title(f'{col} Stripplot')
#     sns.histplot(ax=axes[i][1], x=col, data=test, kde=False, bins=10, palette='GnBu', hue='admit', multiple='dodge')
#     axes[i][1].set_title(f'{col} Displot')
#     sns.boxplot(ax=axes[i][2], x='admit', y=col, data=test, palette='GnBu', hue='admit')
#     axes[i][2].set_title(f'{col} Boxplot')

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(test[num_cols])
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
# for col in num_cols:
#     print(test[col].value_counts())

In [ ]:
# # num_cols.remove(col for col in ['researchExp', 'industryExp', 'internExp', 'journalPubs', 'confPubs', 'admit'])
# print(cat_cols)
# num_cols = ['toeflScore', 'greV', 'greQ', 'cgpa','researchExp', 'industryExp', 'internExp', 'journalPubs', 'confPubs']
# print(num_cols)

In [ ]:
# import statsmodels.api as sm
# def calculate_vif(data):
#     vif_df = pd.DataFrame(columns = ['Feature', 'VIF'])
#     x_var_names = X.columns
#     for i in range(0, x_var_names.shape[0]):
#         y = X[x_var_names[i]]
#         x = X[x_var_names.drop([x_var_names[i]])]
#         r_squared = sm.OLS(y,x).fit().rsquared
#         vif = round(1/(1-r_squared),2)
#         vif_df.loc[i] = [x_var_names[i], vif]
#     return vif_df.sort_values(by = 'VIF', axis = 0, ascending=False, inplace=False)

# calculate_vif(X)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
#Define your X and y data and use this to split the data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=420, stratify=y)

In [ ]:
X_test.head()

In [ ]:
test.head()

In [ ]:
import sklearn
# model = xgb.XGBClassifier()
# model.fit(X_train,y_train)
y_pred = model.predict(X)
# print(X)

# from sklearn import metrics
# print(metrics.confusion_matrix(y_test, y_pred))
# print(sklearn.metrics.accuracy_score(y_test,y_pred))

This is end. Pls.

In [ ]:
xgb_params = {'max_depth': range (2, 10, 1), 'n_estimators': [50, 100, 250, 400, 500, 600, 750, 1000],
              'learning_rate': [0.001, 0.0025, 0.005, 0.0075, 0.01],
              'objective': ['binary:hinge', 'binary:logistic', 'binary:logitraw']
}
xgb_clf = xgb.XGBClassifier()
txgb_model = fit(xgb_clf, xgb_params)

In [ ]:
def fit(clf, params, cv=10, X_train=X_train, y_train=y_train):
    grid = GridSearchCV(clf, params, cv=KFold(n_splits=cv), n_jobs=1, verbose=1, return_train_score=True, scoring='accuracy', refit=True) #verbose and n_jobs help us see the computation time and score of a cv. Higher the value of verbose, more the information printed out.
    grid.fit(X_train, y_train)
    return grid

def make_predictions(model, X_test=X_test):
    return model.predict(X_test)

def best_scores(model):
    # print(f'The mean cross validation test score is: {model.cv_results_.mean_test_score}') #for some reason this wasn't working for me even though the attribute exists so lets just leave it.
    print(f'The best parameters are: {model.best_params_}')
    print(f'The best score that we got is: {model.best_score_}')
    return None

def plot_confusion_matrix(y_pred):
    print('00: True Negatives\n01: False Positives\n10: False Negatives\n11: True Positives\n')
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.matshow(conf_matrix, cmap='GnBu', alpha=0.75)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='large')
    plt.xlabel('Predictions', fontsize=14)
    plt.ylabel('Actuals', fontsize=14)
    plt.title('Confusion Matrix', fontsize=14)
    plt.show()
    return None

def check_scores(y_pred):
    print('Precision: %.3f' % precision_score(y_test, y_pred))
    print('Recall: %.3f' % recall_score(y_test, y_pred))
    print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
    print('F1 Score: %.3f' % f1_score(y_test, y_pred))
    print('ROC-AUC Score: %.3f' % roc_auc_score(y_test, y_pred))
    return None

In [ ]:
# lr_params = {'C':[0.001,.009,0.01,.09,1, 1.01, 1.001], 'penalty':['l1', 'l2']} #lasso and ridge regression
# lr_clf = LogisticRegression(solver='saga', max_iter=5000)
# lr_model = fit(lr_clf, lr_params)

In [ ]:
# best_scores(lr_model)

In [ ]:
# lr_y_pred = make_predictions(lr_model)
# check_scores(lr_y_pred)

In [ ]:
# plot_confusion_matrix(lr_y_pred)

In [ ]:
# lr_feature_scores = lr_model.best_estimator_.coef_[0].tolist()
# lr_fi = pd.DataFrame({'Feature': final_features, 'Feature Importance': lr_feature_scores})
# plt.figure(figsize=(10,6))
# sns.barplot(x='Feature Importance', y='Feature', data=lr_fi, palette='GnBu')
# plt.show()

In [ ]:
gnb_params = {'priors': [None], 'var_smoothing': np.logspace(0,-9, num=100)}
gnb_clf = GaussianNB()
gnb_model = fit(gnb_clf, gnb_params)

In [ ]:
best_scores(gnb_model)

In [ ]:
gnb_y_pred = make_predictions(gnb_model)
check_scores(gnb_y_pred)

In [ ]:
plot_confusion_matrix(gnb_y_pred)

In [ ]:
knns_params = {'n_neighbors': list(range(1, 21)), 'weights': ['uniform', 'distance'],
               'metric': ['euclidean', 'manhattan']}
knns_clf = KNeighborsClassifier()
knns_model = fit(knns_clf, knns_params)

In [ ]:
best_scores(knns_model)

In [ ]:
knns_y_pred = make_predictions(knns_model)
check_scores(knns_y_pred)

In [ ]:
plot_confusion_matrix(knns_y_pred)

In [ ]:
y_pred = knns_y_pred

In [ ]:
# use this to output a submission file. Download the submission file from kaggle outputs and upload it on the competition page.
submission = pd.DataFrame({'uid': uids,'admit':y_pred.astype(str)})
filename = 'sample_sumission.csv'

submission.to_csv(filename,index=False)